In [1]:
pip install google-cloud

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install shapely


Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import pandas as pd
import sys
import os

sys.path.append('C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main')

from ActivePyTools.visualizing_tools import *
#from ActivePyTools.grab_data import eval_object_columns

In [2]:
def eval_object_columns(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Evaluate string expressions stored in object-type columns of a DataFrame,
    excluding specific columns like 'txt' and 'direction'. Each string expression
    is evaluated using a safe method that handles errors.

    :param dataframe: Pandas DataFrame with columns containing string expressions.
    :return: DataFrame with the object-type columns evaluated, where possible.

    Example usage:
    >>> df = pd.DataFrame({'A': ['1+1', '2+2'], 'txt': ['ignore', 'ignore']})
    >>> eval_object_columns(df)
       A      txt
    0  2  ignore
    1  4  ignore
    """

    def try_eval(item):
        try:
            return eval(item)
        except Exception:
            return item

    object_columns = [col for col in dataframe.select_dtypes(include=['object']).columns if
                      col not in ['txt', 'direction']]
    for column in object_columns:
        dataframe[column] = dataframe[column].apply(lambda item: try_eval(item))

    return dataframe

In [1089]:
from ActivePyTools.visualizing_tools import *


crop_path = 'C:/Users/19945/Desktop/EAI6080 advanced visualization/project-shelves/XN-main/docs/crop_pics/google'
pic_name = 'Bookshelves_5.jpg_2_1'
temp_df = pd.read_csv('Bookshelves_5_labeled_df.csv', encoding='ISO-8859-1')
evaled_df = eval_object_columns(temp_df)

In [1090]:
evaled_df

,txt,confidence,vertices,boundBox,slopes,font,word_len,direction,center_point,crop_idx,Left,Top,Label
0,Don't,100,"[(1039, 566), (1066, 563), (1067, 576), (1040,...","{'Width': 28, 'Height': 16, 'Left': 1039, 'Top...","(0.111, -13.0, 0.111, -13.0)",13.038405,27.166155,horizontal,"(1053.0, 571.0)","(0, 0)",1039,563.0,0.0
1,Lik,100,"[(1066, 563), (1080, 561), (1081, 574), (1067,...","{'Width': 15, 'Height': 15, 'Left': 1066, 'Top...","(0.143, -13.0, 0.143, -13.0)",13.038405,14.142136,horizontal,"(1073.5, 568.5)","(0, 0)",1066,561.0,0.0
2,AND,100,"[(1112, 307), (1172, 309), (1170, 364), (1110,...","{'Width': 62, 'Height': 57, 'Left': 1110, 'Top...","(-0.033, 27.5, -0.033, 27.5)",55.036352,60.033324,horizontal,"(1141.0, 335.5)","(0, 0)",1110,307.0,0.0
3,FALL,100,"[(1119, 346), (1187, 353), (1185, 373), (1117,...","{'Width': 70, 'Height': 27, 'Left': 1117, 'Top...","(-0.103, 10.0, -0.088, 10.5)",20.597387,68.311769,horizontal,"(1152.0, 359.75)","(0, 0)",1117,346.0,0.0
4,LOVE,100,"[(1119, 502), (1183, 508), (1181, 530), (1117,...","{'Width': 66, 'Height': 28, 'Left': 1117, 'Top...","(-0.094, 11.0, -0.094, 11.0)",22.090722,64.280635,horizontal,"(1150.0, 516.0)","(0, 0)",1117,502.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,the,100,"[(1039, 286), (1034, 324), (1007, 320), (1012,...","{'Width': 32, 'Height': 41, 'Left': 1007, 'Top...","(7.6, -0.148, 7.4, -0.111)",27.230422,37.831923,vertical,"(1023.0, 303.25)","(2, 1)",1007,283.0,1.0
652,from,100,"[(1048, 219), (1040, 281), (1013, 277), (1021,...","{'Width': 35, 'Height': 65, 'Left': 1013, 'Top...","(7.75, -0.148, 7.625, -0.111)",27.230422,62.018176,vertical,"(1030.5, 248.25)","(2, 1)",1013,216.0,1.0
653,Half,100,"[(1056, 159), (1049, 218), (1021, 214), (1028,...","{'Width': 35, 'Height': 63, 'Left': 1021, 'Top...","(8.429, -0.143, 8.429, -0.143)",28.284271,59.413803,vertical,"(1038.5, 186.5)","(2, 1)",1021,155.0,1.0
654,One,100,"[(1063, 103), (1056, 155), (1029, 151), (1035,...","{'Width': 34, 'Height': 56, 'Left': 1029, 'Top...","(7.429, -0.148, 8.667, -0.143)",27.789480,52.407024,vertical,"(1045.75, 127.0)","(2, 1)",1029,99.0,1.0


In [535]:
def draw_text_on_image(text_row: pd.Series, image, padding: int = 10):
    vertices = text_row.vertices
    x_coords, y_coords = zip(*vertices)

    min_x = max(int(min(x_coords) - padding), 0)
    max_x = min(int(max(x_coords) + padding), image.shape[1])
    min_y = max(int(min(y_coords) - padding), 0)
    max_y = min(int(max(y_coords) + padding), image.shape[0])
    # 
    # print(vertices)
    # print(f"{min_x}, {max_x}; {min_y}, {max_y}")
    # print(image.shape)

    cropped_image = image[min_y:max_y, min_x:max_x]

    fig, ax = plt.subplots()
    ax.imshow(cropped_image, extent=[min_x, max_x, min_y, max_y])

    ax.add_patch(draw_rectangle(get_text_data(text_row), color='red'))

    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)

    ax.legend()
    plt.show()

## Check pics output

In [536]:


# img_path = os.path.join(crop_path, pic_name + '.jpg')
# if not os.path.exists(img_path):
#     print("文件不存在：", img_path)
# else:
#     image = cv2.imread(img_path)
#     if image is None:
#         print("无法读取图像，请检查图像格式和完整性：", img_path)
#     else:
#         # 如果图像读取成功，继续你的处理
#         draw_text_on_image(text_row, image, padding=100)


In [1000]:
id = 609

In [1140]:
id += 1
print(id + 2)
text_row = evaled_df.iloc[id]
print(text_row.txt)
row, col = text_row.crop_idx
img_path = os.path.join(crop_path, pic_name + '.jpg')

draw_text_on_image(text_row, cv2.imread(img_path), padding=100)

658


IndexError: single positional indexer is out-of-bounds

In [1103]:

# 修改对应行的 'label' 列值为 0
evaled_df.loc[id, 'Label'] = 0

# 如果你想查看改变是否成功
print(evaled_df.loc[id])
# print(evaled_df.iloc[id-20:id])


txt                                                             *
confidence                                                    100
vertices         [(604, 137), (603, 151), (580, 150), (581, 136)]
boundBox        {'Width': 24, 'Height': 15, 'Left': 580, 'Top'...
slopes                               (14.0, -0.043, 14.0, -0.043)
font                                                    23.021729
word_len                                                14.035669
direction                                                vertical
center_point                                       (592.0, 143.5)
crop_idx                                                   (2, 1)
Left                                                          580
Top                                                         136.0
Label                                                         0.0
Name: 619, dtype: object


# Check Similarity

In [797]:
import difflib

difflib.SequenceMatcher(None, 'REFUCES', 'REFUGEE').ratio()

0.7142857142857143

In [1141]:
# 假设你的 DataFrame 叫做 evaled_df，你已经进行了一些修改
# 现在你想把这些修改保存到原始文件中

# 指定文件路径，这里使用与读取相同的路径和文件名
file_path = 'Bookshelves_5_labeled_df.csv'

# 保存 DataFrame 到 CSV，覆盖原始文件
evaled_df.to_csv(file_path, index=False, encoding='ISO-8859-1')
